# Работа с "Наташей" (Домашняя работа 5)

#### Сперва все установим и загрузим тексты с лекции

In [2]:
!pip install natasha

    100% |████████████████████████████████| 778kB 26.9MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 33.7MB/s ta 0:00:01


In [3]:
from natasha import NamesExtractor

In [5]:
extractor = NamesExtractor()

In [11]:
with open('sents.txt', 'r') as file:
    sents = file.readlines()

#### Посмотрим, как себя ведет себя Natasha без правил и всего такого

Опробуем на первых 10 строках нашей выборки, чтобы не было быльшой выдачи на Git'e

In [138]:
%%time

extractor('\n'.join(sents[:10]))

CPU times: user 360 ms, sys: 0 ns, total: 360 ms
Wall time: 357 ms


Matches('Напомним, что премьер-министр России Дмитрий Медведев подписал изменения в Правилах дорожного движения.\n\nВ секте Асахары «министр» Ниими слыл страшным человеком.\n\nС тайной полицией Асахары был связан еще один из семи этапированных накануне смертников - Кийохиде Хаякава, «министр созидания».\n\nПо словам Косуги, накануне «министр» Хаякава в личной беседе поблагодарил его за помощь «Аум» и попросил сделать кое-что для «спасения человечества», а именно - убить шефа полиции Японии.\n\nКак только что сообщило немецкое государственное информационное агентство "Deutsche Welle", видный немецкий политик от СДПГ, в прошлом премьер-министр земли Бранденбург Маттиас Платцек, в интервью радиостанции "Deutschlandfunk" в пятницу 16 марта заявил, что по его сведениям лаборатория, которая занималась исследованиями боевых отравляющих веществ в советское время находилась на территории нынешнего Узбекистана.\n\nМежду тем вице-премьер РМ по евроинтеграции Юрий Лянкэ заявил накануне на румынском канале Digi24, что объединение с Румынией - тема, которая становится все более важной в Республике Молдова.\n\nПремьер-министр Израиля Биньямин Нетаньяху и мэр Иерусалима Нир Баркат приняли решение создать правительственную комиссию под руководством министра регионального сотрудничества Цахи Ханегби с участием всех соответствующих сторон для разработки решения вопроса о муниципальных налогах на недвижимость, принадлежащую христианским Церквам и не являющуюся молельными домами.\n\nВ МИД Израиля «НГР» сообщили, что в рамках дискуссии между администрацией главы правительства и муниципалитетом Иерусалима по этому вопросу премьер-министр и мэр Иерусалима договорились о том, что профессиональная группа во главе с министром Ханегби, в которую войдут представители мэрии израильской столицы, а также министерств финансов, иностранных дел и внутренних дел выработает согласованное решение относительно муниципальных налогов с церковной недвижимости, не являющейся молельными домами.\n\nКроме того, в соответствии с поступившей от глав Церквей просьбой начать переговоры о вопросах, связанных с продажей земельных участков в Иерусалиме, премьер-министр Нетаньяху поручил министру Ханегби изучить этот вопрос.\n\nПремьер-министр Великобритании Тереза Мэй в качестве «жесткого ответа» в связи с покушением на двойного агента Сергея Скрипаля и его дочь Юлию вышлет из страны 23 российских дипломата.\n',
        [Match([MorphTagToken('Дмитрий',
                              [37, 44),
                              'RU',
                              'I',
                              [Form('дмитрий',
                                    Grams(NOUN,Name,anim,masc,nomn,sing))]),
                MorphTagToken('Медведев',
                              [45, 53),
                              'RU',
                              'I',
                              [Form('медведев',
                                    Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))])],
               [37, 53)),
         Match([MorphTagToken('Кийохиде',
                              [253, 261),
                              'RU',
                              'I',
                              [Form('кийохида',
                                    Grams(NOUN,Name,anim,datv,femn,sing)),
                               Form('кийохида',
                                    Grams(NOUN,Name,anim,femn,loct,sing)),
                               Form('кийохид',
                                    Grams(NOUN,Name,anim,loct,masc,sing))])],
               [253, 261)),
         Match([MorphTagToken('Хаякава',
                              [262, 269),
                              'RU',
                              'I',
                              [Form('хаякава',
                                    Grams(NOUN,Sgtm,femn,inan,nomn,sing)),
                               Form('хаякава',
                                    Grams(NOUN,Name,anim,masc,nomn,sing)),
                               Form('хаякавый', Grams(ADJS,

#### Теперь попробуем написать правила и посмотрим на наш результат.

In [28]:
from yargy import Parser, rule, or_
from yargy.predicates import gram, normalized
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.tokenizer import MorphTokenizer 
from IPython.display import display

In [117]:
Person = fact(
    'Person',
    ['position', 'name', 'place']
)
Name = fact(
    'Name',
    ['first', 'last', 'patronymic']
)

POSITION = morph_pipeline([
    'премьер министр',
    'премьер-министр',
    'президент',
    'министр',
    'министр обороны',
    'министр иностранных дел',
    'мэр',
    'премьер',
    'канцлер',
    'министр финансов',
    'министр регионального развития',
    'губернатор',
    'глава оргкомитета',
    'глава'])

NAME = or_(
    rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    ),
    gram('Patr').interpretation(
        Name.patronymic.inflected()
    ).optional()
).interpretation(
        Name
),
    rule(
    gram('Name').repeatable().interpretation(
        Name.last.inflected())
).interpretation(
        Name
))

PLACE = rule(
        gram('Geox').interpretation(
            Person.place.normalized()
).optional()
)


PERSON = rule(
    POSITION.interpretation(
        Person.position.normalized()
    ),
    PLACE,
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)

parser = Parser(PERSON)

#### Запишем все, что сможем найти в список, чтобы померить длинну (нам надо больше 50), а также выведем результат поиска

In [139]:
%%time
result = []

for sent in sents:
    for match in parser.findall(sent):
        matches.append(match.fact)

CPU times: user 1.66 s, sys: 4 ms, total: 1.66 s
Wall time: 1.66 s


#### Сколько же мы всего нашли?

In [119]:
len(result)

88

#### Отлично, для проверки посмотрим 

In [136]:
for item in result[:5]:
    print(
    '''
    First name:\t{}
    Second name:\t{}
    Patronomic:\t{}
    Position:\t{}
    Place:\t{}
    ________________
    '''.format(item.name.first,
               item.name.last,
               item.name.patronymic,
               item.position,
               item.place))


    First name:	дмитрий
    Second name:	медведев
    Patronomic:	None
    Position:	премьер-министр
    Place:	россия
    ________________
    

    First name:	None
    Second name:	израиль биньямин нетаньяха
    Patronomic:	None
    Position:	премьер-министр
    Place:	None
    ________________
    

    First name:	None
    Second name:	нетаньяха
    Patronomic:	None
    Position:	премьер-министр
    Place:	None
    ________________
    

    First name:	None
    Second name:	тереза
    Patronomic:	None
    Position:	премьер-министр
    Place:	великобритания
    ________________
    

    First name:	None
    Second name:	алексис ципрас
    Patronomic:	None
    Position:	премьер-министр
    Place:	None
    ________________
    
